In [1]:
import re
!pip install transformers
from transformers import pipeline
import pandas as pd

In [4]:
df=pd.read_excel(r'C:\Users\y.kumar.mohan.bade\Desktop\Yash\Internal Model Enhancements\Summarization\Experiment_output_4_Titles_v2.xlsx')

In [5]:
df.columns

Index(['JFG', 'Job Family', 'UUID', 'Prediction', 'Confidence', 'Title',
       'Attribute', 'Clean Attribute'],
      dtype='object')

In [6]:
#df_part1=df.head(2)
### Appending all clean Attributes in one list 
empty_list=[]
for i in df['Clean Attribute']:
    empty_list.append(i)
print(empty_list)

["Understanding of requirements to integrate new software applications into IAM tools and understand basic mechanics of how existing software applications are integrated into IAM provisioning tools.  Evaluate user provisioning aspects of new clinical and business projects, assists in defining new workflows in IAM tools, assists in creating and executing test scripts for workflows.  For systems and software applications in scope for IAM Team, reconcile discrepancies between access rights assigned and access rights required for users to perform job duties.  Use IAM tools to manage life cycle of user accounts in scope for job responsibilities and use manual provisioning methods where necessary.  Assist in gathering access-related information relative to information security audits or investigations, and privacy investigations.  Run reports or provide data related to IAM activities when called upon to do so.  Assist IAM Engineers in troubleshooting issues with IAM tools and processes.  Und

In [12]:
# import and initialize the tokenizer and model from the checkpoint
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

checkpoint = "sshleifer/distilbart-cnn-12-6"

tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

In [91]:
# extract the sentences from the document
import nltk
nltk.download('punkt')
sentences = nltk.tokenize.sent_tokenize(df['Clean Attribute'][3])

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\y.kumar.mohan.bade\AppData\Roaming\nltk_data.
[nltk_data]     ..
[nltk_data]   Package punkt is already up-to-date!


In [29]:

# find the max tokens in the longest sentence
max([len(tokenizer.tokenize(sentence)) for sentence in sentences])

220

In [15]:

# max tokens including the special tokens
tokenizer.model_max_length 

1024

In [16]:

# max tokens excluding the special tokens
tokenizer.max_len_single_sentence 

1022

In [17]:

# number of special tokens
tokenizer.num_special_tokens_to_add() 

2

In [18]:
sentences

['Understanding of requirements to integrate new software applications into IAM tools and understand basic mechanics of how existing software applications are integrated into IAM provisioning tools.',
 'Evaluate user provisioning aspects of new clinical and business projects, assists in defining new workflows in IAM tools, assists in creating and executing test scripts for workflows.',
 'For systems and software applications in scope for IAM Team, reconcile discrepancies between access rights assigned and access rights required for users to perform job duties.',
 'Use IAM tools to manage life cycle of user accounts in scope for job responsibilities and use manual provisioning methods where necessary.',
 'Assist in gathering access-related information relative to information security audits or investigations, and privacy investigations.',
 'Run reports or provide data related to IAM activities when called upon to do so.',
 'Assist IAM Engineers in troubleshooting issues with IAM tools a

In [20]:
len(sentences)

283

In [92]:

# initialize
length = 0
chunk = ""
chunks = []
count = -1
for sentence in sentences:
  count += 1
  combined_length = len(tokenizer.tokenize(sentence)) + length # add the no. of sentence tokens to the length counter

  if combined_length  <= tokenizer.max_len_single_sentence: # if it doesn't exceed
    chunk += sentence + " " # add the sentence to the chunk
    length = combined_length # update the length counter

    # if it is the last sentence
    if count == len(sentences) - 1:
      chunks.append(chunk.strip()) # save the chunk
    
  else: 
    chunks.append(chunk.strip()) # save the chunk
    
    # reset 
    length = 0 
    chunk = ""

    # take care of the overflow sentence
    chunk += sentence + " "
    length = len(tokenizer.tokenize(sentence))
len(chunks)

6

In [93]:
chunks

["building technical skills through a tailored entry-level development program, working with industry professionals. Get alerts to jobs like this, to your inbox. Thrive in fast-paced environment and can shift priorities effortlessly. Put our customers' needs into everything we do. Offer competitive compensation and benefits packages, challenging projects, company-wide events, co-workers you will actually like working with, and a sense of community you won't find anywhere. Manage the scheduling of pickup and delivery appointments for client shipments. Assist to resolve Accounts Receivable and Accounts Payable issues. Provide operational support for Account Executives by prioritizing between standard daily duties and ad hoc requests. Work with Account Executives to provide world class customer service to clients. Leverage operational support to batch up processes and continually improve the efficiency of overall book of business. Manage relationships within Echo and with key carriers. En

In [22]:
### Checking some chunks
[len(tokenizer.tokenize(c)) for c in chunks]

[997, 982, 997, 1000, 989, 547]

In [23]:
### checking some tokens
[len(tokenizer(c).input_ids) for c in chunks]

[999, 984, 999, 1002, 991, 549]

In [24]:
### checking length with special tokens
sum([len(tokenizer(c).input_ids) for c in chunks])

5524

In [94]:
# inputs to the model.
inputs = [tokenizer(chunk, return_tensors="pt") for chunk in chunks]

#Output 

In [95]:
for input in inputs:
  output = model.generate(**input,max_length=350, min_length=90)
  print(tokenizer.decode(*output, skip_special_tokens=True))

 Work with Account Executives to provide world class customer service to clients. Enjoy working in a team environment but follow through on own deliverables on a daily basis. Thrive in fast-paced environment and can shift priorities effortlessly. Enjoy competitive compensation and benefits packages, challenging projects, company-wide events, co-workers you will actually like working with, and a sense of community you won't find anywhere. Minimum 2-3 years experience in logistics and/or transportation.
 Must be able to handle stress caused in meeting deadlines with tight scheduling requirements. Must excel in time management, prioritizing deadlines, and communicating with Management daily if deadlines need to be extended. Must cause or push forward incremental improvements to internal procedures and programs and work productively with Corporate colleagues in all facets of finding and implementing operation's efficiencies and cost cutting measures. Proficiency with all MS Word programs (

In [96]:
text_1=" Work with Account Executives to provide world class customer service to clients. Enjoy working in a team environment but follow through on own deliverables on a daily basis. Thrive in fast-paced environment and can shift priorities effortlessly. Enjoy competitive compensation and benefits packages, challenging projects, company-wide events, co-workers you will actually like working with, and a sense of community you won't find anywhere. Minimum 2-3 years experience in logistics and/or transportation.Must be able to handle stress caused in meeting deadlines with tight scheduling requirements. Must excel in time management, prioritizing deadlines, and communicating with Management daily if deadlines need to be extended. Must cause or push forward incremental improvements to internal procedures and programs and work productively with Corporate colleagues in all facets of finding and implementing operation's efficiencies and cost cutting measures. Proficiency with all MS Word programs (e. g. Excel, Outlook) and ability to quickly learn warehouse system and transportation systems.Develops and maintains successful relationships with local vendors, freight forwarders, custom brokers, local pickup and delivery companies, regional trucking companies, line haul providers. Ensures compliance with all government related programs specific to mode and direction of related shipments (i. e. customs, security, other import and export government agencies that regulate international trade). Establishes and maintains all necessary procedures to assure a clean, properly organized, safe and healthy environment for all personnel. Familiar with Cargowise software Monitors Daily Reports and Dashboard Creates Invoices for billing purposes.Logistics Coordinator tracks, traces, and updates the status of shipments on a daily basis. Responsible for continuous Core 100 carrier development at all times. Review PTAs and driver's requests for time off or home time in planning loads for same-day or next-day dispatching. Maintain frequent contact with assigned drivers, monitoring call-offs, fatigue concerns and equipment issues. Handle urgent situations such as the need to recover truck, freight, or driver, traffic, weather conditions that may prevent on-time delivery.ROAR coordinates, manages and tracks shipments from point of origin to final destination, while delivering best in class customer service & support. Must follow customer rules and requirements in areas such as dress code, access, hours, etc. Handle urgent situations such as the need to recover truck, freight, or driver, traffic, weather conditions or other unforeseen circumstances that may prevent on-time delivery.Maintain an adequate pool of independent contractors to facilitate deliveries by hosting open house events.Negotiate with vendors to secure best pricing, favorable terms, volume discounts and pricing on all material and/or service purchases. Develop, track, and maintain program budgets. Create, align, and execute procurement, cost reduction, and efficiency strategies. Maintain inventory and reporting process to enable balancing of program assets. Assign carriers to move shipments based on shipment information. Booking & scheduling all shipments within the assigned Account Executive / team."

In [97]:
len(text_1)

3288